# Importing the relevant libraries
### Importing pandas library
### Importing Beautifulsoup for webscraping

In [1]:
import pandas as pd
import bs4 as bs
import numpy as np
import requests as req
print("Relevant libraries imported!!!")

Relevant libraries imported!!!


### Request for fetching the linkedin page

In [2]:
WebPageResult = req.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

### Fetching the table contents of the wikipedia page in to a beautiful soup variable

In [3]:
bsvar = bs.BeautifulSoup(WebPageResult.text)
canadaDataTable = bsvar.find('table',class_='wikitable sortable')

### Defining the Pandas Data Frame Structure

In [5]:
column_names = ['PostalCode','Borough','Neighborhood']
CanadaDataFrame = pd.DataFrame(columns=column_names)
CanadaDataFrame.head()

,PostalCode,Borough,Neighborhood


### Logic of Populating the Dataframe from the Web scraped wikipedia page

In [6]:
columnIdentifier = 0
for column in canadaDataTable.find_all('td'):
    columnIdentifier = columnIdentifier + 1
    if columnIdentifier == 1:
        PostalCode = column.text
    elif columnIdentifier == 2:
        Borough = column.text 
    else:
        columnIdentifier = 0
        if Borough != 'Not assigned':  
            Neighbourhood = column.text
            Neighbourhood = Neighbourhood[:-1]
            if Neighbourhood == 'Not assigned':
                Neighbourhood = Borough
            CheckPostalCode = CanadaDataFrame[CanadaDataFrame['PostalCode'] == PostalCode]
            if CheckPostalCode['Neighborhood'].empty:                
                CanadaDataFrame = CanadaDataFrame.append({'PostalCode': PostalCode,
                                      'Borough' : Borough,
                                      'Neighborhood' : Neighbourhood}, ignore_index=True)
            else:
                ExistingPostNeigh = CheckPostalCode.iloc[0,2]
                Neighbourhood = ExistingPostNeigh + "," + Neighbourhood
                CanadaDataFrame.loc[CanadaDataFrame['PostalCode'] == PostalCode,['Neighborhood']] = Neighbourhood

# Final Shape of the Dataframe answer to the Question 1 of the assigment

In [7]:
CanadaDataFrame.shape

(103, 3)

# Question 2 Answer Starts from here Adding Lattitude and Longitude from CSV File

### Fetching the csv file

In [8]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print("file loaded")

file loaded


### Dataframe creation from csv and setting the index of the dataframe to postalcode

In [12]:
Latlongdf = pd.read_csv("Geospatial_Coordinates.csv")
Latlongdf = Latlongdf.set_index(['Postal Code'])
Latlongdf .head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


#### Creation of a list of the various postalcode for running a for loop

In [13]:
PostalCodeList = Latlongdf.index.values
PostalList = PostalCodeList.tolist()

### Iterating over the list of postal codes to create and fetching the Lattitude and longitude and adding it to the new Dataframe

In [14]:
CanadaDataFrame['latitude'] = np.nan
CanadaDataFrame['longitude'] = np.nan
for postalcode in PostalList:
    PostalLatitude = Latlongdf.loc[postalcode,"Latitude"]
    Postallongitude = Latlongdf.loc[postalcode,"Longitude"]
    CanadaDataFrame.loc[CanadaDataFrame['PostalCode'] == postalcode,['latitude']] = PostalLatitude
    CanadaDataFrame.loc[CanadaDataFrame['PostalCode'] == postalcode,['longitude']] = Postallongitude

# Answer to Question 2 of the Week 3 assignment new Dataframe with all the relevant values

In [15]:
CanadaDataFrame

,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


# Response to Question 3 in the Assignment

### Fetching those rows of the Dataframe which contain Keyword TORONTO in Borough

In [17]:
Torontodf = CanadaDataFrame[CanadaDataFrame['Borough'].str.contains("Toronto")]
Torontodf.head()

,PostalCode,Borough,Neighborhood,latitude,longitude
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [18]:
print('The dataframe has {} boroughs where name contains the keyword Toronto'.format(
        len(Torontodf['Borough'].unique())
    )
)

The dataframe has 4 boroughs where name contains the keyword Toronto


# importing few more libraries

In [24]:
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import folium
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes
#!conda install -c conda-forge folium
#!pip install folium

### Fetching the lattitude and longitude of the Toronto city using the geopy package

In [25]:
address = 'Toronto City, Canada'

geolocator = Nominatim(user_agent="can_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.7189883, -79.44157.


#### Create a map of Toronto City with neighborhoods superimposed on top.
#### One Assumption has been made that all the Borough which are having Toronto in their name will be near about Toronto city

In [22]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Torontodf['latitude'], Torontodf['longitude'], Torontodf['Borough'], Torontodf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Defining the Foursquare Credentials for exploring the Neighbourhoods where the borough name has toronto

In [26]:
CLIENT_ID = 'SZEK3UGUTCP2XMYNTNUTHWGKEHOWV1JJSOTX3FQB41CLY3HW' # your Foursquare ID
CLIENT_SECRET = 'RBAD01ASRUKBUI3PU4AMXA1PGMTDKVPFCGUFXLEN1TON0X4W' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SZEK3UGUTCP2XMYNTNUTHWGKEHOWV1JJSOTX3FQB41CLY3HW
CLIENT_SECRET:RBAD01ASRUKBUI3PU4AMXA1PGMTDKVPFCGUFXLEN1TON0X4W


From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [27]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Beow Function has been written which will be exploring each of the neighbourhoods with the borough name having Toronto keyword

In [29]:
LIMIT = 100 # 
radius = 500 # define radius


In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = req.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Calling the above created function to fetch all the 

In [34]:
Borough_Toronto_venues = getNearbyVenues(names=Torontodf['Neighborhood'],
                                   latitudes=Torontodf['latitude'],
                                   longitudes=Torontodf['longitude']
                                  )

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city


### Analyzing the Total Venues retrurned by the Foursquar API

In [35]:
print(Borough_Toronto_venues.shape)
Borough_Toronto_venues.head()

(1700, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront,Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront,Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


### Grouping the Venues returned for each neighbourhood

In [36]:
Borough_Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton,Exhibition Place,Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",16,16,16,16,16,16
"Cabbagetown,St. James Town",46,46,46,46,46,46
Central Bay Street,88,88,88,88,88,88
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15


### Number of Unique Categories which are found

In [37]:
print('There are {} uniques categories.'.format(len(Borough_Toronto_venues['Venue Category'].unique())))

There are 239 uniques categories.


## Analyzing the neighbourhood using one hot encoding

In [50]:
# one hot encoding
Borough_Toronto_onehot = pd.get_dummies(Borough_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Borough_Toronto_onehot['Neighbor'] = Borough_Toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [Borough_Toronto_onehot.columns[-1]] + list(Borough_Toronto_onehot.columns[:-1])
Borough_Toronto_onehot = Borough_Toronto_onehot[fixed_columns]

print(Borough_Toronto_onehot.shape)
Borough_Toronto_onehot.head()

(1700, 240)


,Neighbor,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Finding the Frequency of the Category for Each Neighborhood

In [51]:
Borough_Toronto_grouped = Borough_Toronto_onehot.groupby('Neighbor').mean().reset_index()
Borough_Toronto_grouped

,Neighbor,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,...,0.00000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000
1,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.045455
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.052632
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,"Cabbagetown,St. James Town",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Central Bay Street,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.011364,...,0.00000,0.00,0.011364,0.000000,0.011364,0.000000,0.011364,0.000000,0.00,0.011364
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00000,0.00,0.060000,0.000000,0.000000,0.030000,0.010000,0.000000,0.00,0.000000
8,Christie,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Church and Wellesley,0.011494,0.011494,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.011494,...,0.00000,0.00,0.000000,0.011494,0.000000,0.011494,0.000000,0.011494,0.00,0.011494


### Similar to lab defining a function for sorting the most common venues in descending Order

In [53]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Creating a dataframe to show top 10 Venues for Each Neighbourhood/PostalCode

In [54]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Borough_Toronto_sorted = pd.DataFrame(columns=columns)
Borough_Toronto_sorted['Neighborhood'] = Borough_Toronto_grouped['Neighbor']
for ind in np.arange(Borough_Toronto_grouped.shape[0]):
    Borough_Toronto_sorted.iloc[ind, 1:] = return_most_common_venues(Borough_Toronto_grouped.iloc[ind, :], num_top_venues)

Borough_Toronto_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,Bar,Steakhouse,American Restaurant,Cosmetics Shop,Burger Joint,Restaurant,Hotel
1,Berczy Park,Coffee Shop,Cocktail Bar,Café,Beer Bar,Steakhouse,Bakery,Seafood Restaurant,Farmers Market,Cheese Shop,Italian Restaurant
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Breakfast Spot,Café,Yoga Studio,Burrito Place,Performing Arts Venue,Stadium,Caribbean Restaurant,Restaurant,Bar
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Pizza Place,Brewery,Skate Park,Burrito Place,Butcher,Restaurant,Recording Studio,Comic Shop,Spa
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Sculpture Garden,Boat or Ferry,Bar,Plane,Coffee Shop


### Clusturing Neighborhood Venues onthe basis of Venue Catrgory

In [59]:
# set number of clusters
kclusters = 4

Borough_Toronto_grouped_clustering = Borough_Toronto_grouped.drop('Neighbor', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Borough_Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

### Adding the Cluster Labels Against the Top Famous Venues

In [60]:
# add clustering labels
Borough_Toronto_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Borough_Toronto_merged = Torontodf

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Borough_Toronto_merged = Borough_Toronto_merged.join(Borough_Toronto_sorted.set_index('Neighborhood'), on='Neighborhood')

Borough_Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Theater,Breakfast Spot,Café,Restaurant,Mexican Restaurant,Brewery
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Ramen Restaurant,Theater,Fast Food Restaurant,Bubble Tea Shop,Italian Restaurant
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Hotel,Restaurant,Breakfast Spot,Cosmetics Shop,Bakery,Italian Restaurant,Gastropub,Pizza Place
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Other Great Outdoors,Health Food Store,Neighborhood,Trail,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Café,Beer Bar,Steakhouse,Bakery,Seafood Restaurant,Farmers Market,Cheese Shop,Italian Restaurant


In [61]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Borough_Toronto_merged['latitude'], Borough_Toronto_merged['longitude'], Borough_Toronto_merged['Neighborhood'], Borough_Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining Cluster

### CLUSTER1

In [62]:
Borough_Toronto_merged.loc[Borough_Toronto_merged['Cluster Labels'] == 0, Borough_Toronto_merged.columns[[1] + list(range(5, Borough_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Park,Bakery,Pub,Theater,Breakfast Spot,Café,Restaurant,Mexican Restaurant,Brewery
9,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Ramen Restaurant,Theater,Fast Food Restaurant,Bubble Tea Shop,Italian Restaurant
15,Downtown Toronto,0,Café,Coffee Shop,Hotel,Restaurant,Breakfast Spot,Cosmetics Shop,Bakery,Italian Restaurant,Gastropub,Pizza Place
19,East Toronto,0,Pub,Other Great Outdoors,Health Food Store,Neighborhood,Trail,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
20,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Café,Beer Bar,Steakhouse,Bakery,Seafood Restaurant,Farmers Market,Cheese Shop,Italian Restaurant
24,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Café,Middle Eastern Restaurant,Sandwich Place,Burger Joint,Indian Restaurant,Spa,Japanese Restaurant,Bar
25,Downtown Toronto,0,Grocery Store,Café,Park,Nightclub,Convenience Store,Baby Store,Diner,Coffee Shop,Restaurant,Italian Restaurant
30,Downtown Toronto,0,Coffee Shop,Café,Thai Restaurant,Bar,Steakhouse,American Restaurant,Cosmetics Shop,Burger Joint,Restaurant,Hotel
31,West Toronto,0,Pharmacy,Supermarket,Bakery,Park,Music Venue,Fast Food Restaurant,Liquor Store,Middle Eastern Restaurant,Café,Bar
36,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Italian Restaurant,Café,Scenic Lookout,Sporting Goods Shop,Brewery,Fried Chicken Joint,Pizza Place


### CLUSTER 2

In [63]:
Borough_Toronto_merged.loc[Borough_Toronto_merged['Cluster Labels'] == 1, Borough_Toronto_merged.columns[[1] + list(range(5, Borough_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,1,Tennis Court,Playground,Concert Hall,Dog Run,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


### CLUSTER 3

In [65]:
Borough_Toronto_merged.loc[Borough_Toronto_merged['Cluster Labels'] == 2, Borough_Toronto_merged.columns[[1] + list(range(5, Borough_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,2,Park,Swim School,Bus Line,Yoga Studio,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


### CLUSTER 4

In [66]:
Borough_Toronto_merged.loc[Borough_Toronto_merged['Cluster Labels'] == 3, Borough_Toronto_merged.columns[[1] + list(range(5, Borough_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,3,Ice Cream Shop,Garden,Yoga Studio,Dog Run,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
